In [ ]:
# import os
# import shutil
# import json

# def copy_files_from_names(file_names, source_folder, dest_folder):
#     for file in file_names:
#         name = file+'.npz'
#         source_file_path = os.path.join(source_folder, name)
#         if os.path.exists(source_file_path):
#             dest_file_path = os.path.join(dest_folder, name)
#             shutil.copy(source_file_path, dest_file_path)

# def copy_first_n_files(source_folder, dest_folder, n):
#     files = os.listdir(source_folder)
#     sorted_files = sorted(files)
#     for file_name in sorted_files[:n]:
#         source_file_path = os.path.join(source_folder, file_name)
#         dest_file_path = os.path.join(dest_folder, file_name)
#         shutil.copy(source_file_path, dest_file_path)
# copy_first_n_files("data", "data-10000",10000)

# with open("recoat_train_val_test.json") as f:
#     j = json.load(f)

# for key in j:
#     copy_files_from_names(j[key], "data", f"data-{key}")


In [ ]:
import sys
sys.path.append("../")

In [ ]:
import os
import numpy as np

def get_max_displacement_in_folder(folder):
    files = os.listdir(folder)
    maxes = []
    bad_idx = []
    for i, file in enumerate(files):
        name = os.path.join(folder, file)
        data = np.load(name)
        maxval = np.max(np.abs(data["disp"][:,2]))
        maxes.append(maxval)
        print(i, end="\r")
        if maxval > 0.5:
            bad_idx.append(i)
    maxes = np.array(maxes)
    return maxes, bad_idx
    
maxes, bad = get_max_displacement_in_folder("../data-train")
maxes_te, bad_te = get_max_displacement_in_folder("../data-test")
maxes_val, bad_val = get_max_displacement_in_folder("../data-val")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4,3),dpi=300)
plt.hist(maxes[maxes<.5],100)
plt.xlabel("Maximum z-Displacement Magnitude")
plt.ylabel("Number of shapes")
plt.title("Displacement < 1")
plt.show()

In [ ]:

import numpy as np
import importlib
from models_gpu import * # Modified
from coarsen import *
import torch
import torch.optim as opt
import visualize
from visualize import *

In [ ]:
import load_graphs
# import parse_netfabb_results
import evaluate
importlib.reload(load_graphs)
# importlib.reload(parse_netfabb_results)
importlib.reload(evaluate)
from load_graphs import *

In [ ]:
# maxes = []
# bad_idx = []
# for i, data in enumerate(train_dataset):
#     maxval = torch.max(torch.abs(data.y))
#     maxes.append(maxval)
#     print(i, end="\r")
#     if maxval > 1:
#         bad_idx.append(i)
# maxes = np.array(maxes)

In [ ]:
device=torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = NetFabbDataset("../data-train", npz_mode=True, N=3, factor=8, replace_edges=True)
test_dataset = NetFabbDataset("../data-test", npz_mode=True, N=3, factor=8, replace_edges=True)
val_dataset = NetFabbDataset("../data-val", npz_mode=True, N=3, factor=8, replace_edges=True)


bad_names = [train_dataset.files[i] for i in bad]
bad_names_te = [test_dataset.files[i] for i in bad_te]
bad_names_val = [val_dataset.files[i] for i in bad_val]

for name in bad_names:
    train_dataset.files.remove(name)
for name in bad_names_te:
    test_dataset.files.remove(name)
for name in bad_names_val:
    val_dataset.files.remove(name)


train_loader = NetFabbDataLoader(train_dataset, shuffle=True, batch_size=4)
test_loader = NetFabbDataLoader(test_dataset, shuffle=True, batch_size=4)
val_loader = NetFabbDataLoader(val_dataset, shuffle=True, batch_size=4)

In [ ]:
conv_info = get_default_conv_info("EdgeConv")
conv_info["dims"] = [128,128]
model = GraphUNet(num_layers=3, num_channels=128, conv_info=conv_info, mlp_dims=[256,256,256]).to(device)


# conv_info = get_default_conv_info("GCNConv")
# model = GraphUNet(num_layers=3, num_channels=128, conv_info=conv_info, mlp_dims=[256,256,256]).to(device)


# model = JustConvNet(num_convs=6, conv_dims=[128,128], channels=128, mlp_dims = [256,256,256]).to(device)

print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
lr = 0.001
epochs = 50
opt = optim.Adam(params=model.parameters(), lr=lr)
lossfun = nn.MSELoss().to(device)
all_losses = []
val_losses = []

for epoch in range(epochs+1):
    losses = []
    for i, data in enumerate(train_loader):
        loss = lossfun(model(data, device).reshape(-1,1), data.y.reshape(-1,1).to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
        print(f"Batch {i+1}/{len(train_loader)}: Loss = {loss.item():.6e}, Avg = {np.mean(np.array(losses)):.6e}          ",end="\r")
        del data
        torch.cuda.empty_cache() 

    if 1: # 0 == (epoch%5):
        print(f"Epoch: {epoch}/{epochs}, Loss: {np.mean(np.array(losses)):.6e}                             ")# (max {np.max(np.array(losses)):.6e}, median {np.median(np.array(losses)):.6e})                ")
    all_losses.append(np.array(losses))

    model.eval()
    losses = []
    for i, data in enumerate(test_loader):
        loss = lossfun(model(data, device).reshape(-1,1), data.y.reshape(-1,1).to(device))
        losses.append(loss.item())
        del data
        torch.cuda.empty_cache() 
    if 1: # 0 == (epoch%5):
        print(f"   Val. Loss: {np.mean(np.array(losses)):.6e}                              ")
    val_losses.append(np.array(losses))
    model.train()

    np.savez("mdl-new-losses.npz", all_losses=np.array(all_losses), val_losses=np.array(val_losses))
    torch.save(model, "mdl-new.pth")
    
    
    

In [ ]:
# np.savez("model-gnn-600k-f8-50-v2-losses.npz", all_losses=np.array(all_losses), val_losses=np.array(val_losses))
# torch.save(model, "model-gnn-600k-f8-50-v2.pth")
#model = torch.load("../model-300k-25.pth").to(device)

# np.savez("unet-new-losses.npz", all_losses=np.array(all_losses), val_losses=np.array(val_losses))
# torch.save(model, "unet-new.pth")

In [ ]:
#data = train_dataset[2]
data = test_dataset[70]
i = -1#len(test_dataset)//2+3

#data = test_dataset[idx_te[i]]
#print(vals["te"][idx_te[i]])

# i = 0
# data = train_dataset[idx_tr[i]]
# print(vals["tr"][idx_tr[i]])

verts = data.x.detach().numpy()
gt = data.y.detach().numpy()
pred = model(data, device).cpu().detach().numpy()
error = pred.flatten() - gt.flatten()

plot_fields(verts, [gt, pred, error], ["Ground Truth", "Prediction", "Pred - GT"], cmap="jet")


In [ ]:
model2 = JustConvNet(num_convs=5)

In [ ]:
lr = 0.001
epochs = 50
opt = optim.Adam(params=model2.parameters(), lr=lr)
lossfun = nn.MSELoss()

for epoch in range(epochs+1):
    losses = []
    for i, data in enumerate(train_loader):
        loss = lossfun(model2(data).reshape(-1,1), data.y.reshape(-1,1))
        opt.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
        print(f"Shape {i+1}: Loss = {loss.item():.6e}",end="\r")
    if 1: # 0 == (epoch%5):
        print(f"Epoch: {epoch}/{epochs}, Loss: {np.mean(np.array(losses)):.6e}")

In [ ]:
#data = train_dataset[2]
data = test_dataset[0]

verts = data.x.detach().numpy()
gt = data.y.detach().numpy()
pred = model2(data).detach().numpy()
error = pred.flatten() - gt.flatten()

plot_fields(verts, [gt, pred, error], ["Ground Truth", "Prediction", "Pred - GT"], cmap="jet")

In [ ]:
importlib.reload(evaluate)
import evaluate
from evaluate import *

In [ ]:
dsets = dict(tr=train_dataset, te=test_dataset)
titles = dict(tr="Training",te="Testing")

vals = eval_model_multiple(model, dsets, device)
plot_boxes(vals,titles, lims=[-1,1])

In [ ]:
idx_tr = np.argsort(vals["tr"])
idx_te = np.argsort(vals["te"])

In [ ]:
vals2 = eval_model_multiple(model2, dsets)
plot_boxes(vals2,titles)

In [ ]:
np.median(vals["tr"])

In [ ]:
np.median(vals["te"])

In [ ]:
#data = train_dataset[2]
for i in range(40,48):
    data = test_dataset[i]

    verts = data.x.detach().numpy()
    gt = data.y.detach().numpy()
    pred = model(data).detach().numpy()
    error = pred.flatten() - gt.flatten()

    plot_fields(verts, [gt, pred, error], ["Ground Truth", "Prediction", "Pred - GT"], cmap="jet")
